<a href="https://colab.research.google.com/github/pravsels/DistilLM/blob/main/HF_Training_Script.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets transformers accelerate peft bitsandbytes

In [2]:
! apt install git-lfs

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git-lfs is already the newest version (3.0.2-1ubuntu0.2).
0 upgraded, 0 newly installed, 0 to remove and 33 not upgraded.


In [23]:
from huggingface_hub import notebook_login
notebook_login()

In [24]:
import transformers
from datasets import load_dataset
import math
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments

In [25]:
wikitext_dataset = load_dataset('mediciresearch/manimation')

In [26]:
wikitext_dataset['train'], wikitext_dataset['eval'] = wikitext_dataset['train'].train_test_split(.1).values()

wikitext_dataset

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 920
    })
    eval: Dataset({
        features: ['text'],
        num_rows: 103
    })
})

In [27]:
wikitext_dataset['train'][11]

{'text': '<s>[INST] Generate accurate and correct ManimCE Python code for the animation requested by the user. Here is the user\'s request: Can you create an educational animation that illustrates the concept of the Taylor series expansion by showing the mathematical function e^x along with its Taylor approximation up to the x cubed term? First, display the function \'f(x) = e^x\' on the screen, then write out its approximate Taylor series expansion starting with \'1 + x\' through to \'x cubed over 3 factorial\', followed by an ellipsis to imply the series continues indefinitely. Explain with text that this is an approximation around the point x=0 and highlight the polynomial terms specifically. Also mention that adding more terms of the series increases the approximation accuracy.\n\nAfter the text explanations, could you show a graph with the real function \'e^x\' plotted, labeled accordingly, and demonstrate the polynomial approximation on the same graph with a different color to di

In [28]:
wikitext_dataset['eval'][11]

{'text': '<s>[INST] Generate accurate and correct ManimCE Python code for the animation requested by the user. Here is the user\'s request: I would like an animated video explaining the process of solving a basic definite integral using the power rule. The video should start by introducing the integral of x cubed from 0 to 2, then demonstrate step-by-step how to apply the power rule to find the indefinite integral. Please include on-screen text that states "Apply the power rule of integration" followed by the general formula for the power rule. Next, show how to substitute the specific function and limits into the formula, simplify the resulting expression, and evaluate the integral at the upper and lower bounds. Conclude with the numerical value of the definite integral and then clear the screen to end with showing just the final result. The presentation should be clear and paced with brief pauses to allow the viewer to follow along. [/INST] \\n from manim import *\nfrom manim_physics

In [50]:
base_model = 'deepseek-ai/deepseek-coder-6.7b-instruct'
# model_name = 'smallcloudai/Refact-1_6B-fim'
# using the same tokenizer that was used for training the model
tokenizer = AutoTokenizer.from_pretrained(base_model, use_fast=True)

In [30]:
def tokenize_func(examples):
  return tokenizer(examples['text'])

In [31]:
tokenized_dataset = wikitext_dataset.map(tokenize_func,
                                         batched=True,
                                         num_proc=8,
                                         remove_columns=['text'])

Map (num_proc=8):   0%|          | 0/920 [00:00<?, ? examples/s]

Map (num_proc=8):   0%|          | 0/103 [00:00<?, ? examples/s]

In [32]:
print(tokenized_dataset)
print(tokenized_dataset['train'][0])
print(tokenized_dataset['eval'][0])

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 920
    })
    eval: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 103
    })
})
{'input_ids': [32013, 27, 82, 29, 58, 24607, 60, 9338, 387, 10370, 285, 2907, 2797, 308, 4402, 13003, 2974, 327, 254, 17181, 13159, 457, 254, 2664, 13, 4451, 317, 254, 2664, 6, 82, 3092, 25, 14919, 340, 4058, 2594, 274, 14711, 17181, 327, 523, 344, 3521, 254, 3036, 1089, 254, 10264, 320, 405, 1371, 61, 17, 11, 473, 1371, 28, 15, 276, 1371, 28, 17, 11, 1242, 245, 5487, 285, 5561, 15492, 276, 3278, 254, 3036, 30, 6067, 11, 3417, 245, 841, 280, 7626, 257, 365, 5487, 4290, 11, 4976, 254, 10264, 285, 254, 3036, 11, 285, 463, 254, 4889, 28895, 631, 429, 254, 1223, 13, 303, 6, 67, 833, 245, 3443, 12, 1946, 12, 9218, 17181, 1064, 254, 7626, 257, 285, 254, 10264, 417, 10253, 1019, 11, 5482, 457, 254, 13846, 11, 930, 254, 3036, 8155, 279, 11, 285, 6339, 3685, 267, 1691, 631, 13, 82

In [33]:
# setting context length of out inputs
# block_size = tokenizer.model_max_length
block_size = 1024     # context len of input

In [34]:
def group_text_func(examples):
  # for every key ('train', 'test' etc.), concatenate the texts
  concat_examples = {key: sum(examples[key], []) for key in examples.keys()}
  total_len = len(concat_examples[list(examples.keys())[0]])
  print(list(examples.keys()))
  total_len = (total_len // block_size) * block_size

  # split by chunks of block_size
  result = {
      key: [text[i : i + block_size] for i in range(0, total_len, block_size)] for key, text in concat_examples.items()
  }
  result['labels'] = result['input_ids'].copy()

  return result

In [35]:
lm_dataset = tokenized_dataset.map(group_text_func,
                                   batched=True,
                                   batch_size=128,
                                   num_proc=8)
lm_dataset

Map (num_proc=8):   0%|          | 0/920 [00:00<?, ? examples/s]

['input_ids', 'attention_mask']
['input_ids', 'attention_mask']
['input_ids', 'attention_mask']
['input_ids', 'attention_mask']
['input_ids', 'attention_mask']
['input_ids', 'attention_mask']
['input_ids', 'attention_mask']
['input_ids', 'attention_mask']


Map (num_proc=8):   0%|          | 0/103 [00:00<?, ? examples/s]

['input_ids', 'attention_mask']['input_ids', 'attention_mask']

['input_ids', 'attention_mask']['input_ids', 'attention_mask']

['input_ids', 'attention_mask']
['input_ids', 'attention_mask']
['input_ids', 'attention_mask']['input_ids', 'attention_mask']



DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 570
    })
    eval: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 58
    })
})

In [36]:
print('input ids : ', lm_dataset['train'][0]['input_ids'][:5])
print('attention mask : ', lm_dataset['train'][0]['attention_mask'][:5])
print('label ids : ', lm_dataset['train'][0]['labels'][:5])

input ids :  [32013, 27, 82, 29, 58]
attention mask :  [1, 1, 1, 1, 1]
label ids :  [32013, 27, 82, 29, 58]


In [37]:
from peft import LoraConfig

peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    target_modules=["q_proj", "k_proj"],
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
)

In [38]:
model = AutoModelForCausalLM.from_pretrained(model_name,
                                             device_map="auto",
                                             load_in_8bit=True)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [39]:
model.add_adapter(peft_config,
                  adapter_name="manim_adapter")

In [40]:
import numpy as np
def count_trainable_parameters(model):
    model_parameters = filter(lambda p: p.requires_grad, model.parameters())
    params = sum([np.prod(p.size()) for p in model_parameters])
    total_params = sum([np.prod(p.size()) for p in model.parameters()])
    return params/total_params

count_trainable_parameters(model) * 100

0.49533656825843125

In [41]:
training_args = TrainingArguments(
  f"{model_name}-finetuned-manimation",
  evaluation_strategy = "epoch",
  # num_train_epochs=10,
  learning_rate=2e-5,
  per_device_train_batch_size=1,
  per_device_eval_batch_size=1,
  gradient_accumulation_steps=16,
  weight_decay=0.01,
  push_to_hub=True
)

In [42]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_dataset['train'],
    eval_dataset=lm_dataset['eval']
)

In [43]:
trainer.train()

Epoch,Training Loss,Validation Loss
0,No log,1.146811
1,No log,1.134922
2,No log,1.129658


TrainOutput(global_step=105, training_loss=1.0954900832403274, metrics={'train_runtime': 1158.4368, 'train_samples_per_second': 1.476, 'train_steps_per_second': 0.091, 'total_flos': 6.855764169719808e+16, 'train_loss': 1.0954900832403274, 'epoch': 2.95})

In [44]:
eval_results = trainer.evaluate()
print(f"Perplexity score : {math.exp(eval_results['eval_loss']):.2f}")

Perplexity score : 3.09


In [46]:
model.save_pretrained('./')

/usr/local/lib/python3.10/dist-packages/transformers/integrations/peft.py:389: FutureWarning: The `active_adapter` method is deprecated and will be removed in a future version.
  warnings.warn(


In [47]:
trainer.push_to_hub()

/usr/local/lib/python3.10/dist-packages/transformers/integrations/peft.py:389: FutureWarning: The `active_adapter` method is deprecated and will be removed in a future version.
  warnings.warn(


events.out.tfevents.1708136177.1e0c7eb2a20a.9226.0:   0%|          | 0.00/8.28k [00:00<?, ?B/s]

Upload 7 LFS files:   0%|          | 0/7 [00:00<?, ?it/s]

events.out.tfevents.1708135942.1e0c7eb2a20a.7728.0:   0%|          | 0.00/4.81k [00:00<?, ?B/s]

events.out.tfevents.1708135770.1e0c7eb2a20a.5539.0:   0%|          | 0.00/4.18k [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/134M [00:00<?, ?B/s]

events.out.tfevents.1708136372.1e0c7eb2a20a.10253.0:   0%|          | 0.00/5.96k [00:00<?, ?B/s]

events.out.tfevents.1708137548.1e0c7eb2a20a.10253.1:   0%|          | 0.00/354 [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/4.66k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/pravsels/deepseek-coder-6.7b-instruct-finetuned-manimation/commit/d0b0e0c5d462911313f874ddb89fb738b956dd13', commit_message='End of training', commit_description='', oid='d0b0e0c5d462911313f874ddb89fb738b956dd13', pr_url=None, pr_revision=None, pr_num=None)

In [51]:
model_name = 'pravsels/deepseek-coder-6.7b-instruct-finetuned-manimation'
tokenizer = AutoTokenizer.from_pretrained(base_model, use_fast=True)

In [56]:
model = AutoModelForCausalLM.from_pretrained(model_name,
                                             device_map="auto",
                                             load_in_4bit=True)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [57]:
gen_context_len = 512

input_text = "write manim code to transition a blue circle to a red square. please write code only, no explanation."
inputs = tokenizer(input_text, return_tensors="pt").to(model.device)
outputs = model.generate(**inputs, max_length=gen_context_len)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:32021 for open-end generation.
/usr/local/lib/python3.10/dist-packages/bitsandbytes/nn/modules.py:226: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(f'Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.')


write manim code to transition a blue circle to a red square. please write code only, no explanation.


```python
from manim import *

class SquareToCircle(Scene):
    def construct(self):
        circle = Circle()  # create a circle
        circle.set_fill(BLUE, opacity=0.5)  # set the color and transparency
        self.play(Create(circle))  # show the circle on screen

        square = Square()  # create a square
        square.rotate(PI / 4)  # rotate a certain amount
        square.set_fill(RED, opacity=0.5)  # set the color and transparency
        self.play(Create(square))  # show the square on screen

        self.play(TransformAround(circle, square))  # interpolate circle to square
        self.wait()
```

This code creates a blue circle and a red square, then transitions the blue circle to the red square.

```python
from manim import *

class SquareToCircle(Scene):
    def construct(self):
        circle = Circle()  # create a circle
        circle.set_fill(BLUE, opacity=0.5)